# Implementation of Ensemble Kalman Inversion (EKI) based on (Iglesias, Law, and Stewart, 2013)

This python notebook provides a step by step implementation of the Ensemble Kalman Inversion (EKI) introduced in (Iglesias, Law, and Stewart, 2013). The EKI is one numerical method for inverse problems, which aims to recover one or more parameters in a model given some measurements. The EKI algorithm consists of the following steps:

1. Generating an initial ensemble (a set of initial guesses for the parameter values)
2. Evaluating forward maps, i.e. what are the corresponding model outputs for the given parameters
3. Computing the covariances and the Kalman gain
4. Updating the ensemble until it satisfies a convergence criterion


We start by presenting two methods for generating the initial ensemble. The function initialize_uniform_ensemble considers the case where the initial ensemble is drawn from a uniform distribution with lower bound $lb$, and upper bound $ub$. On the other hand,  initialize_normal_ensemble draws the initial ensemble from a normal distribution with mean $\mu$ and standard deviation $\sigma$.

In [1]:
## generate the initial ensemble based on information known about the parameter's distribution

# initialize ensemble based on uniform distribution
# inputs: 
# N_ens - number of ensembles 
# N_params - number of parameters to be estimated
# lb - lower bound
# ub - upper bound
# outputs:
# ini_ens - initial set of ensembles
# ens_mean - mean of the initial set of ensembles

def initialize_uniform_ensemble(N_ens, N_params, lb = 0, ub = 1):
    ini_ens = np.random.uniform(lb,ub,(N_ens,N_params))
    ens_mean = np.mean(ini_ens, axis = 0)
    return ini_ens, ens_mean

# initialize ensemble based on normal distribution
# inputs: 
# N_ens - number of ensembles 
# N_params - number of parameters to be estimated
# mu - mean
# sigma - standard deviation
# outputs:
# ini_ens - initial set of ensembles
# ens_mean - mean of the initial set of ensembles
def initialize_normal_ensemble(N_ens, N_params, mu = 0, sigma = 1):
    ini_ens = np.random.normal(mu, sigma, (N_ens,N_params))
    ens_mean = np.mean(ini_ens, axis = 0)
    return ini_ens, ens_mean

The next block of code evaluates forward maps. That is, we map the parameters to the model outputs, which typically corresponds to the measurements. To illustrate this, we present two examples. The first example comes from the inverse problem of recovering the parameters $(a,b)$ for a given function
\begin{equation}
f(x) = ae^{bx}, x\in[0,1].
\end{equation}

That is, given measurements $f(x_1),\dots,f(x_k)$ at the data points $x_1,\dots,x_k\in[0,1]$, we aim to estimate the parameters $(a,b)$. Correspondingly, for the parameters $(\hat{a},\hat{b})$, the forward map associated to this inverse problem is given by $\hat{f}(x) = \hat{a}e^{\hat{b}x}$, and returns the values $\hat{f}(x_1),\dots,\hat{f}(x_k)$.

The second example deals with recovering the source term $s$ of the ODE with homogeneous Dirichlet boundary conditions

\begin{equation}
\begin{aligned}
-p'' + p &= s,\\
p(0)=p(1)&=0,
\end{aligned}
\end{equation}

given pointwise measurements $p(x_1),\dots,p(x_k)$ for $x_1,\dots,x_k \in (0,1)$. Thus, the parameters we aim to recover are the values $s(x_1),\dots,s(x_k)$. For this example, we consider a forward map which solves for $p$ given $s$ via the two-point flux approximation (TPFA) finite volume method. For simplicity, we work under the assumption that $x_1,\dots,x_k$ are equally spaced. 

In [2]:
# compute the forward map of one parameter 
# inputs:
# curr_ens_mem - current ensemble (set of parameters)
# testCase - test case number
# output:
# forward_map_val - output of the forward model based on the current parameter set
def evaluate_forward_map(curr_ens_mem, testCase = 1): 
    if testCase == 1:
        forward_map_val = curr_ens_mem[0]*np.exp(curr_ens_mem[1]*x_points)
    else:
        if testCase == 2:
            curr_rhs = process_rhs(curr_ens_mem, xL, xR, Nx)
            curr_sol = np.transpose(np.linalg.solve(A_fd,curr_rhs))
            forward_map_val = curr_sol[0][1:Nx+1]
    return forward_map_val

## compute the forward map of the entire ensemble, and also the mean of the forward mapped values
# inputs: 
# curr_ens - current ensemble family (collection of set of parameters)
# output:
# curr_ens_evals - output of the forward model for the ensemble family 
def compute_ens_forward(curr_ens): 
    nDataPts = len(np.transpose(curr_ens))
    nEnsembles = len(curr_ens)
    curr_ens_evals = np.zeros((nEnsembles,Nx))
    for ensNum in range(nEnsembles):
        curr_ens_member = curr_ens[ensNum][:]
        curr_ens_evals[ensNum][:] = evaluate_forward_map(curr_ens_member,testNum)
    curr_ens_eval_mean = np.mean(curr_ens_evals, axis = 0)
    return curr_ens_evals, curr_ens_eval_mean

The next step is to compute the covariances. As an input, it requires the current ensemble members and their corresponding forward maps. We design the function to require the means as input (alternatively, the mean may also be computed within the function itself). 

In [3]:
## compute the covariance matrices

# inputs:
# curr_ens - current family of ensembles
# curr_ens_mean - mean of curr_ens
# curr_ens_evals - forward map (evaluations) of the current ensemble family
# curr_ens_eval_mean - mean of curr_ens_evals

# outputs:
# CGG- sample covariance of the forward mapped data
# CpG - sample covariance between the forward mapped data and the ensemble

def compute_covariances(curr_ens,curr_ens_mean,curr_ens_evals,curr_ens_eval_mean):
    nEnsembles = len(curr_ens)
    nParamValues = len(np.transpose(curr_ens))
    out_size = len(np.transpose(curr_ens_eval_mean)) #size of the output of a forward map
    CGG = np.zeros((out_size,out_size))
    CpG = np.zeros((nParamValues,out_size))
    for ensNum in range(nEnsembles):
        curr_ens_eval = curr_ens_evals[ensNum][:]
        GG = np.add(curr_ens_eval, -1*curr_ens_eval_mean)
        GG = [GG]
        GGt = np.transpose(GG)
        GGprod = np.matmul(GGt,GG)
        CGG = np.add(CGG, GGprod)
        curr_ens_member = curr_ens[ensNum][:]
        pp = np.add(curr_ens_member, -1*curr_ens_mean)
        pp = [pp]
        pGprod = np.matmul(np.transpose(pp),GG)
        CpG = np.add(CpG,pGprod)
    CGG = 1/(nEnsembles-1)*CGG
    CpG = 1/(nEnsembles-1)*CpG
    return CGG,CpG

def compute_Gamma(p_data,noise_lvl):
    Gamma = noise_lvl*noise_lvl/4*p_data*p_data
    Gamma = np.diag(Gamma.flatten())
    return Gamma

After computing all the required components, we can now update the ensemble. The computed components that are needed are the current ensemble, its forward maps, and the covariance matrices. Together with these, we also require the measured data and the assumed noise level. 

In [4]:
## update the ensemble

# inputs:
# curr_ens - current family of ensembles
# curr_ens_evals - forward map (evaluations) of the current ensemble family
# CGG- sample covariance of the forward mapped data
# CpG - sample covariance between the forward mapped data and the ensemble
# Gamma - covariance of the data
# p_noisy - measured data / data provided
# noise_lvl - assumed level of noise

# output:
# new_ensemble - ensemble obtained after an EKI update

def update_ensemble(curr_ens,curr_ens_evals,CGG,CpG,Gamma,p_noisy,noise_lvl):
    nEnsembles = len(curr_ens)
    nOutputs = len(p_noisy)
    nParamValues = len(np.transpose(curr_ens))
    new_ensemble = np.zeros((nEnsembles,nParamValues))
    Kalman_lhs = np.add(CGG,Gamma)
    for ensNum in range(nEnsembles):
        eta_ens = np.zeros((1,nOutputs))
        curr_ens_member = curr_ens[ensNum][:]
        curr_ens_eval = curr_ens_evals[ensNum][:]
        for paramNum in range(nOutputs):
            eta_ens[0][paramNum] = np.random.normal(0,p_noisy[paramNum]*noise_lvl)
        Kalman_rhs = np.add(np.transpose(p_noisy),eta_ens)
        Kalman_rhs = np.add(Kalman_rhs, -1*curr_ens_eval)
        Kalman_gain = np.linalg.solve(Kalman_lhs,np.transpose(Kalman_rhs))
        Kalman_gain = np.matmul(CpG,Kalman_gain)
        new_ensemble[ensNum][:] = np.add(curr_ens_member, np.transpose(Kalman_gain))

    return new_ensemble


Finally, we provide the EKI algorithm. It terminates upon reaching the maximum number of iterations, or when the difference between the forward map and the data is less than the noise level. The main inputs are the number of ensembles and the maximum number of iterations. Following these, we require to specify the data points, their noise level, and which test case they came from.  

In [5]:
#coḿpute the parameter values via the EKI method

#inputs: 
# nEnsembles - number of ensembles
# maxIter - maximum number of iterations / EKI updates
# p_noisy - measured data / data provided
# noise_lvl - assumed level of noise
# testNum - number of the test case

#outputs: 
# current_ensemble - final (updated) ensemble after EKI has converged or maxIter has been reached
# ens_mean - mean of current_ensemble

def compute_parameter_values(nEnsembles,maxIter,p_noisy,noise_lvl,testNum):
    if testNum == 1:
        nParams = 2
        lb = 1
        ub = 4
    elif testNum == 2:
        nParams = len(p_noisy)
        lb = 0
        ub = 1
    # compute the covariance of the data
    Gamma = compute_Gamma(p_noisy,noise_lvl)
    # initialize the ensemble
    ini_ens, ens_mean = initialize_uniform_ensemble(nEnsembles, nParams, lb, ub)
    # compute the forward maps
    [ens_evals, ens_eval_mean] =  compute_ens_forward(ini_ens)
    # compute the covariance matrices
    CGG,CpG = compute_covariances(ini_ens,ens_mean,ens_evals,ens_eval_mean)
    current_ensemble = copy_matrix(ini_ens)
    ens_mean = np.mean(current_ensemble, axis = 0)
    numIter = 0
    while numIter<maxIter:
        [ens_evals, ens_eval_mean] =  compute_ens_forward(current_ensemble)
        err_vec = np.add(ens_eval_mean,-1*np.transpose(p_noisy))
        err_now = np.linalg.norm(np.add(ens_eval_mean,-1*np.transpose(p_noisy)))/np.linalg.norm(p_noisy)
        if err_now<noise_lvl:
            #print(err_now)
            #print(numIter)
            break
        CGG,CpG = compute_covariances(current_ensemble,ens_mean,ens_evals,ens_eval_mean)
        updated_ensemble =  update_ensemble(current_ensemble, ens_evals, CGG,CpG, Gamma, p_noisy, noise_lvl)
        current_ensemble = copy_matrix(updated_ensemble)
        ens_mean = np.mean(current_ensemble, axis = 0)
        numIter += 1
        if numIter == maxIter:
            print (err_now)
            print ("Ensemble did not converge after maximum number of iterations")
    [ens_evals, ens_eval_mean] =  compute_ens_forward(current_ensemble)
    #plt.plot(x_points,ens_eval_mean)
    return current_ensemble, ens_mean

In [6]:
## code for copying an array or matrix in Python, so that the variable doesn't get overwritten
def copy_matrix(my_matrix):
    dim1 = len(my_matrix)
    dim2 = len(np.transpose(my_matrix))
    copied_matrix = np.zeros((dim1,dim2))
    for i in range(dim1):
            copied_matrix[i][:] = my_matrix[i][:]
    return copied_matrix

Functions needed for generating test case 2

In [7]:
import numpy as np
import math
import random
from matplotlib import pyplot as plt

def generate_matrix(xL=0, xR = 1, Nx = 20): #generates an (Nx+2) X (Nx+2) finite volume matrix for approximating -p''+p on (xL,xR)
    nTotal = Nx+2 #Nx interior points, 2 boundary points
    dx = (xR-xL)/(Nx+1)
    A = np.zeros((nTotal,nTotal)) #initialize the (Nx+2) X (Nx+2) matrix for discretizing the problem 
    A[0][0]=1 #Dirichlet BC
    A[Nx+1][Nx+1] = 1
    for i in range(1,Nx+1):
        #diffusion / second-derivative component
        A[i][i-1] = -1/dx
        A[i][i] = 2/dx 
        A[i][i+1] = -1/dx
        #reaction component
        A[i][i] = A[i][i] + dx
    return A

#function / source term for the right hand side of the PDE 
def rhs_function(x):
    return math.sin(math.pi*x)

#evaluate the function at the Nx interior points in the domain
def generate_rhs(xL=0, xR = 1, Nx = 20, BCl=0, BCr=0):
    nTotal = Nx+2
    dx = (xR-xL)/(Nx+1)
    b = np.zeros((nTotal,1))
    b[0] = BCl
    b[Nx+1] = BCr
    x_points = np.linspace(xL,xR,Nx+2)
    for i in range(1,Nx+1):
        b[i] = rhs_function(x_points[i])
    return b

#process the value of the right hand side for a finite volume scheme (approximate an integral for a FV cell)
def process_rhs(rhs_vals, xL=0, xR = 1, Nx = 20, BCl=0, BCr=0):
    nTotal = Nx+2
    dx = (xR-xL)/(Nx+1)
    b = np.zeros((nTotal,1))
    b[0] = BCl
    b[Nx+1] = BCr
    for i in range(1,Nx+1):
        b[i] = rhs_vals[i-1]*dx
    return b

# Generate data 
For our examples, we obtain the data by evaluating the models at the true parameter values. The latter block of code generates the exact data from the true parameters

In [8]:
def true_param_values(testCase = 1):
    if testCase == 1:
        true_vals = [3,2]
    elif testCase == 2:
        true_vals = generate_rhs(xL,xR,Nx)
        true_vals = true_vals[1:Nx+1]        
    return true_vals
    

def generate_data(Nx,true_vals, xL = 0, xR = 1, testCase = 1):  
    if testCase == 1:
        x_points = np.linspace(xL,xR,Nx)
        p_data = true_vals[0]*np.exp(true_vals[1]*x_points)     
        p_data = np.transpose(p_data)
    elif testCase == 2:
        x_points = np.linspace(xL,xR,Nx+2)
        A_fd = generate_matrix(xL,xR,Nx) 
        b = process_rhs(true_vals,xL,xR,Nx)
        p_data = np.linalg.solve(A_fd,b)
        p_data = p_data[1:Nx+1]
        x_points = x_points[1:Nx+1]
    #plt.plot(x_points,p_data)
    return p_data, x_points

# Adding noise to model data
We now add noise to the data (this will be the data we try to recover RHS from)

In [9]:
def add_noise_to_data(p_data, noise_lvl = 1e-3):
    Nx = len(p_data)
    mu, sigma = 0, noise_lvl # mean and standard deviation
    s = np.random.normal(mu, sigma, Nx) 
    p_noisy = np.zeros((Nx,1))
    for i in range(Nx):
        p_noisy[i] = p_data[i]*(1+s[i])
    return p_noisy,noise_lvl

# Initializing and running the tests


In [12]:
Nx = 15 #number of data points
N_ens = 40 #number of ensembles
maxIter = 20 #maximum number of iterations
noise_lvl = 1e-3
testNum = 1
if testNum == 2:
    xL = 0
    xR = 1
    A_fd = generate_matrix(xL,xR,Nx) 
true_vals = true_param_values(testNum)
p_data, x_points = generate_data(Nx, true_vals, testCase = testNum)
p_noisy, noise_lvl = add_noise_to_data(p_data,noise_lvl)

#plt.plot(x_points,p_noisy)

Run the EKI algorithm and compute the error

In [13]:
converged_ens,converged_mean = compute_parameter_values(N_ens,maxIter,p_noisy,noise_lvl,testNum)
#print(converged_mean)
ens_err = np.add(converged_mean,-1*np.transpose(true_vals))
ens_err = np.abs(ens_err)
err_norm = np.linalg.norm(ens_err)
print(ens_err)
print(err_norm)

[0.00058805 0.00051718]
0.0007831250194518339
